In [1]:
#!pip install pyyaml

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv("GWSI_2024_wells_enriched.csv", low_memory=False)
df = df[df['WELL_DEPTH'] >= 0.5]
# --------------------------------
# Step 1: Parse dates
# --------------------------------
date_cols = ['DRILL_DATE', 'LASTWLDATE']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# --------------------------------
# Step 2: Drop redundant column(s)
# --------------------------------
redundant_cols = ['SUBBASIN_NAME_GWSI']
df.drop(columns=redundant_cols, inplace=True, errors='ignore')

# --------------------------------
# Step 3: Move metadata to separate DataFrame
# --------------------------------
join_field = ['OBJECTID']
metadata_cols = join_field + [
    'SITE_ID', 'REG_ID', 'WELL_TYPE', 'WATER_USE', 'CASE_DIAM',
    'WL_COUNT', 'DRILL_DA_1', 'LSTWLDT_TE', 'SGMC_LABEL', 'UNIT_LINK',
    'MINOR1', 'MINOR2', 'MINOR3', 'ROCK_TYPE', 'NAME_ABBR_1', 'NAME_ABBR_12'
]


metadata = df[metadata_cols].copy()
df.drop(columns=[col for col in metadata_cols if col != 'OBJECTID'], inplace=True, errors='ignore')

# --------------------------------
# Step 4: Coerce numeric filter fields
# --------------------------------
numeric_cols = ['AGE_MIN', 'AGE_MAX']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# --------------------------------
# Step 5: Rename coordinates if needed for clarity
df.rename(columns={'LATITUDE': 'y', 'LONGITUDE': 'x'}, inplace=True)

# --------------------------------
# Step 6: Drop rows missing key Z-values for analysis
value_cols = ['WELL_DEPTH', 'WL_DTW', 'WL_ELEV']
df = df.dropna(subset=value_cols)

# --------------------------------
# Step 7: Save final datasets
df.to_parquet("wells_cleaned_main.parquet", index=False)
metadata.to_parquet("wells_metadata.parquet", index=False)

print("✅ Saved: `wells_cleaned_main.parquet` and `wells_metadata.parquet`")

✅ Saved: `wells_cleaned_main.parquet` and `wells_metadata.parquet`


In [5]:
import json
import yaml

# Define the structured data dictionary based on the user's table
columns_info = [
    ("OBJECTID", "Unique internal identifier (typically from GIS system).", "GWSI original points", "JoinField"),
    ("SITE_ID", "Unique site/well identifier assigned by the agency.", "GWSI original points", "metadata"),
    ("REG_ID", "Regional identifier (e.g., agency or administrative region).", "GWSI original points", "metadata"),
    ("WELL_TYPE", "Type of well (e.g., monitoring, production, irrigation).", "GWSI original points", "metadata"),
    ("LATITUDE", "Latitude in decimal degrees (NAD83).", "GWSI original points", "y-coordinate"),
    ("LONGITUDE", "Longitude in decimal degrees (NAD83).", "GWSI original points", "x-coordinate"),
    ("LAT_NAD83", "Latitude in decimal degrees (NAD83).", "GWSI original points", "y-coordinate"),
    ("LAT_NAD27", "Latitude in decimal degrees (NAD27).", "GWSI original points", "y-coordinate"),
    ("LONG_NAD27", "Longitude in decimal degrees (NAD27).", "GWSI original points", "x-coordinate"),
    ("WELL_ALT", "Elevation of the well site (feet).", "GWSI original points", "z-coordinate"),
    ("WATER_USE", "Primary use of the water (e.g., domestic, municipal, agricultural).", "GWSI original points", "metadata"),
    ("WELL_DEPTH", "Total depth of the well (feet).", "GWSI original points", "value (z-coord)"),
    ("CASE_DIAM", "Diameter of the well casing (inches?).", "GWSI original points", "metadata"),
    ("WL_COUNT", "Number of water level measurements available.", "GWSI original points", "metadata"),
    ("WL_DTW", "Depth to water (distance from land surface to water level).", "GWSI original points", "value (z-coord)"),
    ("WL_ELEV", "Elevation of the water surface (datum-dependent).", "Computed from DTW + alt", "value (z-coord)"),
    ("DRILL_DATE", "Date the well was drilled.", "GWSI original points", "filtering"),
    ("LASTWLDATE", "Date of last water level measurement.", "GWSI original points", "filtering"),
    ("FREQUENCY", "Frequency of measurement or monitoring.", "GWSI original points", "filtering"),
    ("DRILL_DA_1", "Duplicate or alternative drill date (legacy field).", "GWSI original points", "metadata"),
    ("LSTWLDT_TE", "Duplicate or alternative last measurement date (legacy field).", "GWSI original points", "metadata"),
    ("SGMC_LABEL", "Stratigraphic label from state geologic mapping. Surficial Geology", "SGMC Geology AZ", "metadata"),
    ("UNIT_LINK", "Geologic unit code or linking key. Surficial Geology", "SGMC Geology AZ", "metadata"),
    ("UNIT_NAME", "Name of geologic unit intersected. Surficial Geology", "SGMC Geology AZ", "group by"),
    ("AGE_MIN", "Minimum geologic age (e.g., epoch or system). Surficial Geology", "SGMC Geology AZ", "filtering"),
    ("AGE_MAX", "Maximum geologic age. Surficial Geology", "SGMC Geology AZ", "filtering"),
    ("MAJOR1", "Primary lithologic or geologic descriptor. Surficial Geology", "SGMC Geology AZ", "group by"),
    ("MAJOR2", "Secondary lithologic descriptor. Surficial Geology", "SGMC Geology AZ", "group by"),
    ("MAJOR3", "Tertiary lithologic descriptor. Surficial Geology", "SGMC Geology AZ", "group by"),
    ("MINOR1", "Additional lithologic descriptor. Surficial Geology", "SGMC Geology AZ", "metadata"),
    ("MINOR2", "Additional lithologic descriptor. Surficial Geology", "SGMC Geology AZ", "metadata"),
    ("MINOR3", "Additional lithologic descriptor. Surficial Geology", "SGMC Geology AZ", "metadata"),
    ("GENERALIZE", "Generalized geologic classification. Surficial Geology", "SGMC Geology AZ", "group by"),
    ("ROCK_NAME", "Specific rock name at the well. Surficial Geology", "SGMC Geology AZ", "group by"),
    ("ROCK_TYPE", "Numeric or coded rock type category. Surficial Geology", "SGMC Geology AZ", "metadata"),
    ("AQ_NAME", "Name of major aquifer co-located with well.", "ADWR Groundwater Subbasin", "group by"),
    ("NAME_ABBR", "Abbreviated name of basin/subunit.", "Basin polygon layer", "group by"),
    ("BASIN_NAME_1", "Name of AMA or INA if in one.", "AZ_AMA_and_INA", "group by"),
    ("NAME_ABBR_1", "Abbreviated name of basin.", "AZ_AMA_and_INA", "metadata"),
    ("SUBBASIN_NAME", "Subdivision of the hydrologic basin.", "ADWR Groundwater Subbasin", "group by"),
    ("NAME_ABBR_12", "Abbreviated name for subbasin or region.", "ADWR Groundwater Subbasin", "metadata"),
    ("SUBBASIN_NAME_GWSI", "Subbasin name used in GWSI.", "ADWR Groundwater Subbasin", "redundant")
]

# Convert to schema format
schema = [{"name": col, "description": desc, "source": source, "functionality": func}
          for col, desc, source, func in columns_info]

# Save as YAML
yaml_path = r"C:\Users\rl587\PycharmProjects\Depth_to_groundwater\docs\wells_schema.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(schema, f, sort_keys=False)

# Save as JSON
json_path = r"C:\Users\rl587\PycharmProjects\Depth_to_groundwater\docs\wells_schema.json"
with open(json_path, "w") as f:
    json.dump(schema, f, indent=2)

yaml_path, json_path

('C:\\Users\\rl587\\PycharmProjects\\Depth_to_groundwater\\docs\\wells_schema.yaml',
 'C:\\Users\\rl587\\PycharmProjects\\Depth_to_groundwater\\docs\\wells_schema.json')

# GWSI 2024 Wells Enriched – Column Descriptions

This document describes the fields in the `GWSI_2024_wells_enriched` dataset.

| Column Name         | Description                                                             | Source                     | Functionality             |
|---------------------|-------------------------------------------------------------------------|----------------------------|---------------------------|
| `OBJECTID`          | Unique internal identifier (typically from GIS system).                 | GWSI original points       | JoinFiled                 |
| `SITE_ID`           | Unique site/well identifier assigned by the agency.                     | GWSI original points       | metadata                  |
| `REG_ID`            | Regional identifier (e.g., agency or administrative region).            | GWSI original points       | metadata                  |
| `WELL_TYPE`         | Type of well (e.g., monitoring, production, irrigation).                | GWSI original points       | metadata                  |
| `LATITUDE`          | Latitude in decimal degrees (NAD83).                                    | GWSI original points       | y-coordinate              |
| `LONGITUDE`         | Longitude in decimal degrees (NAD83).                                   | GWSI original points       | x-coordinate              |
| `LAT_NAD83`         | Latitude in decimal degrees (NAD83).                                    | GWSI original points       | y-coordinate              |
| `LAT_NAD27`         | Latitude in decimal degrees (NAD27).                                    | GWSI original points       | y-coordinate              |
| `LONG_NAD27`        | Longitude in decimal degrees (NAD27).                                   | GWSI original points       | x-coordinate              |
| `WELL_ALT`          | Elevation of the well site (feet).                                      | GWSI original points       | z-coordinate              |
| `WATER_USE`         | Primary use of the water (e.g., domestic, municipal, agricultural).     | GWSI original points       | metadata                  |
| `WELL_DEPTH`        | Total depth of the well (feet).                                         | GWSI original points       | value (z-coord)           |
| `CASE_DIAM`         | Diameter of the well casing (inches?).                                  | GWSI original points       | metadata                  |
| `WL_COUNT`          | Number of water level measurements available.                           | GWSI original points       | metadata                  |
| `WL_DTW`            | Depth to water (distance from land surface to water level).             | GWSI original points       | value (z-coord)           |
| `WL_ELEV`           | Elevation of the water surface (datum-dependent).                       | Computed from DTW + alt    | value (z-coord)           |
| `DRILL_DATE`        | Date the well was drilled.                                              | GWSI original points       | filtering                 |   
| `LASTWLDATE`        | Date of last water level measurement.                                   | GWSI original points       | filtering                 |
| `FREQUENCY`         | Frequency of measurement or monitoring.                                 | GWSI original points       | filtering                 |
| `DRILL_DA_1`        | Duplicate or alternative drill date (legacy field).                     | GWSI original points       | metadata                  |
| `LSTWLDT_TE`        | Duplicate or alternative last measurement date (legacy field).          | GWSI original points       | metadata                  |
| `SGMC_LABEL`        | Stratigraphic label from state geologic mapping. Surficial Geology      | SGMC Geology AZ            | metadata                  |
| `UNIT_LINK`         | Geologic unit code or linking key. Surficial Geology                    | SGMC Geology AZ            | metadata                  |
| `UNIT_NAME`         | Name of geologic unit intersected. Surficial Geology                    | SGMC Geology AZ            | group by                  |
| `AGE_MIN`           | Minimum geologic age (e.g., epoch or system).  Surficial Geology        | SGMC Geology AZ            | filtering                 |
| `AGE_MAX`           | Maximum geologic age. Surficial Geology                                 | SGMC Geology AZ            | filtering                 |
| `MAJOR1`            | Primary lithologic or geologic descriptor. Surficial Geology            | SGMC Geology AZ            | group by                  |
| `MAJOR2`            | Secondary lithologic descriptor. Surficial Geology                      | SGMC Geology AZ            | group by                  |
| `MAJOR3`            | Tertiary lithologic descriptor. Surficial Geology                       | SGMC Geology AZ            | group by                  |
| `MINOR1`            | Additional lithologic descriptor. Surficial Geology                     | SGMC Geology AZ            | metadata                  |
| `MINOR2`            | Additional lithologic descriptor. Surficial Geology                     | SGMC Geology AZ            | metadata                  |
| `MINOR3`            | Additional lithologic descriptor. Surficial Geology                     | SGMC Geology AZ            | metadata                  |
| `GENERALIZE`        | Generalized geologic classification.  Surficial Geology                 | SGMC Geology AZ            | group by                  |
| `ROCK_NAME`         | Specific rock name at the well. Surficial Geology                       | SGMC Geology AZ            | group by                  |
| `ROCK_TYPE`         | Numeric or coded rock type category. Surficial Geology                  | SGMC Geology AZ            | metadata                  |
| `AQ_NAME`           | Name of major aquifer co-located with well.                             | ADWR Groundwater Subbasin  | group by                  |
| `NAME_ABBR`         | Abbreviated name of basin/subunit.                                      | Basin polygon layer        | group by                  |
| `BASIN_NAME_1`      | Name of AMA or INA if in one.                                           | AZ_AMA_and_INA             | group by                  |
| `NAME_ABBR_1`       | Abbreviated name of basin.                                              | AZ_AMA_and_INA             | metadata                  |
| `SUBBASIN_NAME`     | Subdivision of the hydrologic basin.                                    | ADWR Groundwater Subbasin  | group by                  |
| `NAME_ABBR_12`      | Abbreviated name for subbasin or region.                                | ADWR Groundwater Subbasin  | metadata                  |
| `SUBBASIN_NAME_GWSI`| Subbasin name used in GWSI.                                             | ADWR Groundwater Subbasin  | redundant                 |    
